In [94]:
import os

from sqlalchemy import and_, or_
import pandas as pd
pd.set_option("display.max_rows", None)

from grdb.config import Config
from grdb.database.v1_1_0 import dal as db1     # prod dal
from grdb.database import dal as db2            # test dal
from grdb.database.v1_1_0 import Base as Base1  # prod base
from grdb.database import Base as Base2         # test base

from grdb.database.v1_1_0.models import Sample,Recipe,Author


In [95]:
# Setup old schema (production db) as db1 using read only
config_prefix = "PROD_DATABASE"
config_suffix = "_READ"
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db1.init_db(conf, privileges={"read": True, "write": False, "validate": False})
# Testing Configuration and dal initialization
config_prefix = "DEV_DATABASE"
config_suffix = "_ADMIN" 
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db2.init_db(conf, privileges={"read": True, "write": True, "validate": True})

# 0) Stand up fresh empty db2
ses2 = db2.Session()
ses2.close()
Base2.metadata.drop_all(bind=db2.engine)
Base2.metadata.create_all(bind=db2.engine)

In [96]:
# Get all of the samples with their recipe data
ses1 = db1.Session()
pd.read_sql(
    ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id)
    .statement,
    ses1.bind,
)
# For each row:
# 1) Create Expirment row, use ll 1:1 mapped columns
# 1) Get author info from sample.recipe
# 2) create author row (if not exists)
# 3) Add author id to 

,id,software_name,software_version,primary_sem_file_id,nanohub_userid,experiment_date,material_name,validated,id_1,sample_id,thickness,diameter,length,catalyst,tube_diameter,cross_sectional_area,tube_length,base_pressure,dewpoint,sample_surface_area
0,1,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,1,1,0.2,NaN,NaN,Palladium,55.0,2376.0,760.0,0.015500,None,50.0
1,2,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,2,2,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
2,3,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,3,3,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
3,4,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,4,4,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
4,5,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,5,5,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
5,6,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,6,6,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
6,7,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,7,7,0.2,NaN,NaN,Palladium leaf,50.8,2026.8,762.0,0.015400,None,1500.0
7,8,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,8,8,0.3,NaN,NaN,Nickel,50.8,2026.8,762.0,0.015000,None,75.0
8,9,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,9,9,0.3,NaN,NaN,Nickel,50.8,2026.8,762.0,0.015000,None,75.0
9,10,gresq,1.0.0-dev1,None,NaN,None,Graphene,True,10,10,0.3,NaN,NaN,Nickel,50.8,2026.8,762.0,0.015000,None,22.5


In [97]:
ses1 = db1.Session()
unique_substrate_query = ses1.query(Recipe.catalyst, Recipe.thickness, Recipe.diameter, Recipe.length, Recipe.sample_surface_area).distinct().filter(
        or_(
            Recipe.catalyst != None,
            Recipe.thickness != None,
            Recipe.diameter != None,
            Recipe.sample_surface_area != None,
            Recipe.length != None,
        )
    )

pd.read_sql(
    unique_substrate_query.statement,
    ses1.bind,
)

,catalyst,thickness,diameter,length,sample_surface_area
0,Palladium,0.2,NaN,NaN,50.0
1,Palladium leaf,0.2,NaN,NaN,1500.0
2,Nickel,0.3,NaN,NaN,75.0
3,Nickel,0.3,NaN,NaN,22.5
4,Nickel,NaN,NaN,NaN,NaN
5,Palladium leaf,1.0,NaN,NaN,1500.0
6,Palladium,NaN,127.0,1000.0,NaN
7,Palladium,25.0,NaN,NaN,NaN
8,Platinum,25.0,NaN,NaN,NaN
9,Platinum,200.0,NaN,NaN,NaN


In [98]:
# Test query for furnace data

ses1 = db1.Session()
unique_furnace_query = ses1.query(Recipe.tube_diameter, Recipe.cross_sectional_area, Recipe.tube_length).distinct().filter(
        or_(
            Recipe.tube_diameter != None,
            Recipe.cross_sectional_area != None,
            Recipe.tube_length != None,
        )
    )
pd.read_sql(
    unique_furnace_query.statement,
    ses1.bind,
)

,tube_diameter,cross_sectional_area,tube_length
0,55.0,2376.0,760.0
1,50.8,2026.8,762.0
2,25.4,507.0,NaN
3,40.0,1257.0,NaN
4,28.0,616.0,NaN
5,102.0,8107.0,NaN
6,22.0,380.0,NaN
7,25.4,506.0,1000.0


In [99]:
# Read all unique authors from old schema
from grdb.database.v1_1_0.models import Author

ses1 = db1.Session()
#unique_authors_query = ses1.query(
#        Author.sample_id,Author.first_name, Author.last_name, Author.institution
#    ).filter(Author.sample_id==Sample.id).group_by(Author.sample_id,Author.first_name, Author.last_name)
unique_authors_query = ses1.query(
        Author.first_name, Author.last_name, Author.institution
    ).distinct()
pd.read_sql(
    unique_authors_query.statement,
    ses1.bind,
)

,first_name,last_name,institution
0,Kaihao,Zhang,University of Illinois at Urbana-Champaign
1,Xiaohong,An,Northeastern University
2,Fangze,Liu,Northeastern University
3,Yung Joon,Jung,Northeastern University
4,Swastik,Kar,Northeastern University
5,Doglin,Ma,Peking University
6,Mengxi,Liu,Peking University
7,Teng,Gao,Peking University
8,Cong,Li,Peking University
9,Jingyu,Sun,Peking University


In [100]:
ses1 = db1.Session
authors = ses1.query(
    Author.sample_id, Author.first_name, Author.last_name, Author.institution
).group_by(Author.first_name, Author.last_name)
ses1.query(Sample.id,Sample.nanohub_userid).filter(Sample.id.in_(authors))

In [101]:
ses1 = db1.Session()
# authors = ses1.query(Author.first_name, Author.last_name, Author.institution).distinct()
# TODO: SAMPLE TABLE IS COMPLETELY EMPTY
samples = ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id)
for s in samples:
    print(f"name: {s.Sample.material_name}")
    print(f"{s.Recipe.base_pressure,s.Recipe.catalyst,s.Recipe.thickness,s.Recipe.diameter,s.Recipe.length,s.Recipe.sample_surface_area}")

name: Graphene
(0.0155, 'Palladium', 0.2, None, None, 50.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.0154, 'Palladium leaf', 0.2, None, None, 1500.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 22.5)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', None, None, None, None)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.015, 'Nickel', 0.3, None, None, 75.0)
name: Graphene
(0.016, 'Palladium

In [102]:
# This is a prototype to migrate sem file data
from grdb.database.v1_1_0.models import SemFile, SemAnalysis

ses1 = db1.Session()

sem_file_query = ses1.query(SemFile)
pd.read_sql(
    sem_file_query.statement,
    ses1.bind,
)

,id,sample_id,filename,url,default_analysis_id
0,1,19,KZPd_170926-2_SEM1.tif,https://app.box.com/shared/static/0gcoswatti4d...,None
1,2,19,KZPd_170926-2_SEM2.tif,https://app.box.com/shared/static/tqscru4t48dm...,None
2,3,20,KZPd_170928-3_SEM2.tif,https://app.box.com/shared/static/r4jcbampmdn3...,None
3,4,20,KZPd_170928-3_SEM1.tif,https://app.box.com/shared/static/wcfz6ikrzb46...,None
4,5,21,KZPd_171010-1_SEM1.tif,https://app.box.com/shared/static/8xalvtw1u9vo...,None
5,6,21,KZPd_171010-1_SEM2.tif,https://app.box.com/shared/static/zgu2e631r420...,None
6,7,22,KZPd_171010-2_SEM2.tif,https://app.box.com/shared/static/tp024cn0s7yt...,None
7,8,22,KZPd_171010-2_SEM1.tif,https://app.box.com/shared/static/quqzml0oah67...,None
8,9,23,KZPd_180314-1_SEM1.tif,https://app.box.com/shared/static/5vzum9ze0dqo...,None
9,10,23,KZPd_180314-1_SEM2.tif,https://app.box.com/shared/static/ap53ichjfi1s...,None


In [103]:
# This is a prototype to migrate raman file data
from grdb.database.v1_1_0.models import RamanFile, RamanSet, RamanSpectrum

ses1 = db1.Session()

raman_file_query = ses1.query(RamanFile)
pd.read_sql(
    raman_file_query.statement,
    ses1.bind,
)

,id,sample_id,filename,url,wavelength
0,1,19,KZPd_170926-2_Raman1.txt,https://app.box.com/shared/static/dntoyp6st99k...,532.0
1,2,20,KZPd_170928-2_Raman1.txt,https://app.box.com/shared/static/1i0k14kcgxna...,532.0
2,3,21,KZPd_171010-1_Raman1.txt,https://app.box.com/shared/static/kxqxjevcyhgl...,532.0
3,4,22,KZPd_171010-2_Raman1.txt,https://app.box.com/shared/static/vi52dz5iuvle...,532.0
4,5,27,20180321-1_sample.txt,https://app.box.com/shared/static/wkyyc6if2xq3...,532.0
5,6,32,sample.txt,https://app.box.com/shared/static/n5edqfjw59r9...,532.0
6,7,33,SAMPLE.txt,https://app.box.com/shared/static/av8rs1776813...,532.0
7,8,34,sample.txt,https://app.box.com/shared/static/b8h1g889lspt...,532.0
8,9,35,sample.txt,https://app.box.com/shared/static/mmrrr6r8tq9c...,532.0
9,10,39,sample.txt,https://app.box.com/shared/static/24hz65xtmyxa...,532.0


In [104]:
raman_set_query = ses1.query(RamanSet)
pd.read_sql(
    raman_set_query.statement,
    ses1.bind,
)



,id,nanohub_userid,map_file,sample_id,experiment_date,d_to_g,gp_to_g,d_peak_shift,d_peak_amplitude,d_fwhm,g_peak_shift,g_peak_amplitude,g_fwhm,g_prime_peak_shift,g_prime_peak_amplitude,g_prime_fwhm
0,1,None,False,1,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,None,False,2,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,False,3,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,None,False,4,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,None,False,5,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,None,False,6,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,None,False,7,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,None,False,8,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,None,False,9,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,None,False,10,2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
raman_spectrum_query = ses1.query(RamanSpectrum)
pd.read_sql(
    raman_spectrum_query.statement,
    ses1.bind,
)


,id,set_id,raman_file_id,software_name,software_version,xcoord,ycoord,percent,d_peak_shift,d_peak_amplitude,d_fwhm,g_peak_shift,g_peak_amplitude,g_fwhm,g_prime_peak_shift,g_prime_peak_amplitude,g_prime_fwhm
0,1,19,1,gsaraman,1.3.0-dev0,None,None,100.0,1351.42,0.145354,32.3961,1597.10,0.394144,33.0001,2688.41,1.041490,32.4792
1,2,20,2,gsaraman,1.3.0-dev0,None,None,100.0,1356.97,0.097957,50.0000,1586.84,1.034000,33.0000,2702.44,0.597385,60.0000
2,3,21,3,gsaraman,1.3.0-dev0,None,None,100.0,1347.80,0.217042,50.0000,1587.78,0.300000,60.0000,2683.78,1.007830,35.4183
3,4,22,4,gsaraman,1.3.0-dev0,None,None,100.0,1343.95,0.255433,50.0000,1585.43,0.372445,60.0000,2673.10,0.999487,41.1548
4,5,27,5,gsaraman,1.3.0-dev0,None,None,100.0,1332.40,0.152727,50.0000,1582.85,0.300000,49.0430,2674.65,1.025580,32.0000
5,6,32,6,gsaraman,1.3.0-dev0,None,None,100.0,1300.00,0.177400,50.0000,1589.87,0.506958,33.0000,2677.79,1.022610,32.0000
6,7,33,7,gsaraman,1.3.0-dev0,None,None,100.0,1345.31,0.159572,50.0000,1590.29,0.407543,33.0000,2685.99,1.034590,33.9584
7,8,34,8,gsaraman,1.3.0-dev0,None,None,100.0,1335.09,0.196651,50.0000,1586.87,0.307563,60.0000,2681.00,0.992628,32.0000
8,9,35,9,gsaraman,1.3.0-dev0,None,None,100.0,1343.28,0.181009,50.0000,1586.22,0.300000,58.6357,2682.10,1.031640,32.0000
9,10,39,10,gsaraman,1.3.0-dev0,None,None,100.0,1344.36,0.047787,50.0000,1580.13,0.300000,33.0000,2686.04,0.818165,32.0000


In [106]:
unique_raman_spectrum_query = ses1.query(
        RamanSpectrum.xcoord, RamanSpectrum.ycoord
    ).distinct()
pd.read_sql(
    unique_raman_spectrum_query.statement,
    ses1.bind,
)

,xcoord,ycoord
0,None,None


In [108]:
######################################################################################
# Migration script
# ToDo:
# - TESTING!!!
# - nanohub_userid
# - environment_conditions
# - sem_files + analysis
# - raman_files + analysis
# - software versions
######################################################################################

from grdb.database.v1_1_0.models import Author
from grdb.database.v1_1_0.models import SemFile
from grdb.database.v1_1_0.models import RamanFile
from grdb.database.v1_1_0.models import RamanSet
from grdb.database.v1_1_0.models import RamanSpectrum
from grdb.database.models import Author as Author2
from grdb.database.models import Furnace as Furnace2
from grdb.database.models import Experiment as Experiment2
from grdb.database.models import Recipe as Recipe2
from grdb.database.models import PreparationStep as PreparationStep2
from grdb.database.models import Substrate as Substrate2
from grdb.database.models import EnvironmentConditions as EnvironmentConditions2
from grdb.database.models import SemFile as SemFile2
from grdb.database.models import RamanFile as RamanFile2
from grdb.database.models import RamanAnalysis as RamanAnalysis2



commit = True
limit = 200

from math import isclose

# 0) Stand up fresh empty db2
print("Clearing destination database...")
ses2 = db2.Session()
ses2.close()
Base2.metadata.drop_all(bind=db2.engine)
Base2.metadata.create_all(bind=db2.engine)
print("done.")




print("Adding furnace rows...")
furnaces = unique_furnace_query
new_furnaces = [
    Furnace2(
        tube_diameter =  f.tube_diameter,
        cross_sectional_area = f.cross_sectional_area,
        tube_length = f.tube_length
    ) for f in furnaces
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_furnaces)
print("done.")
       

print("Adding substrate rows...")
substrates = unique_substrate_query
new_substrates = [
    Substrate2(
        catalyst = s.catalyst,
        thickness = s.thickness,
        diameter = s.diameter,
        surface_area = s.sample_surface_area,
        length = s.length
    ) for s in substrates
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_substrates)
print("done.")

print("Adding author rows...")
authors = unique_authors_query
new_authors = [
    Author2(
        first_name = a.first_name,
        last_name = a.last_name,
        institution = a.institution
    ) for a in authors
]

# Submit type of call/ insert/update
if commit:
    # call factory function gives live session from class 
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_authors)
print("done.")

print("Getting old sample data...")
ses1 = db1.Session()
samples = ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id).limit(limit)
oauthors = ses1.query(Author).all()
print("done.")

#####################################################################################
# At this point unique rows for furnace, substrate and Authors have been created
# Now, Loop back through the samples and match rows in furnace, substrate and authors
# Build a map from Experiment id  -> furnace_id, etc. for each sample/experiment id
# Note this assumes that the experiment id will map to the (old) sample id.
# ToDo
# - Add fields to preparation_step
# - Add sem_files
# - Add raman_files
# - Add properties
######################################################################################
ses3 = db2.Session()
new_furnaces = ses3.query(Furnace2).all()
new_substrates = ses3.query(Substrate2).all()
new_authors = ses3.query(Author2).all()
    
furnacemap = {}
substratemap = {}
authormap = {}
tol = 0.01
print ("Matching parent tables...")
for s in samples:
    authormap[s.Sample.id]=[]
    print(f"name: {s.Sample.material_name}")
    #print(f"{s.Recipe.base_pressure,s.Recipe.catalyst}")
    
    #################################################################################
    # Match to a row in the furnace table
    #################################################################################
    for f in new_furnaces:
        #print(f"id: {f.id}")
        match = match1 = match2 = match3 = False
        null = null1 = null2 = null3 = False
        if f.tube_diameter:
            if s.Sample.recipe.tube_diameter:
                if isclose(f.tube_diameter,s.Sample.recipe.tube_diameter,rel_tol=0.001,abs_tol=0.001):
                    match1 = True
        else:
            if not s.Sample.recipe.tube_diameter:
                match1 = True
                null1 = True
        if f.tube_length:
            if s.Sample.recipe.tube_length:
                if isclose(f.tube_length,s.Sample.recipe.tube_length,rel_tol=0.001,abs_tol=0.001):
                    match2 = True
        else:
            if not s.Sample.recipe.tube_length:
                match2 = True
                null2 = True
        if f.cross_sectional_area:
            if s.Sample.recipe.cross_sectional_area:
                if isclose(f.cross_sectional_area,s.Sample.recipe.cross_sectional_area,rel_tol=0.001,abs_tol=0.001):
                    match3 = True
        else:
            if not s.Sample.recipe.cross_sectional_area:
                match3 = True
                null3 = True
        # If there are any non null matches    
        match = match1 and match2 and match3
        if match:
            furnacemap[s.Sample.id] = f.id
            #print(f"match: {s.Sample.id}: {f.id}")
            break
        else:
            furnacemap[s.Sample.id] = None
            
        
    #################################################################################
    # Match to row in Substrate table
    #################################################################################
    for sb in new_substrates:
        match = match1 = match2 = match3 = match4 = match5 = False
        if sb.catalyst:
            if sb.catalyst == s.Sample.recipe.catalyst:
                match1 = True
        else:
            if not s.Sample.recipe.catalyst:
                match1 = True
        if sb.thickness:
            if s.Sample.recipe.thickness:
                if isclose(sb.thickness,s.Sample.recipe.thickness,rel_tol=tol,abs_tol=tol):
                    match2 = True
        else:
            if not s.Sample.recipe.thickness:
                match2 = True
        if sb.diameter:
            if s.Sample.recipe.diameter:
                if isclose(sb.diameter,s.Sample.recipe.diameter,rel_tol=tol,abs_tol=tol):
                    match3 = True
        else:
            if not s.Sample.recipe.diameter:
                match3 = True
        if sb.length:
            if s.Sample.recipe.length:
                if isclose(sb.length,s.Sample.recipe.length,rel_tol=tol,abs_tol=tol):
                    match4 = True
        else:
            if not s.Sample.recipe.length:
                match4 = True
        if sb.surface_area:
            if s.Sample.recipe.sample_surface_area:
                if isclose(sb.surface_area,s.Sample.recipe.sample_surface_area,rel_tol=tol,abs_tol=tol):
                    match5 = True
        else:
            if not s.Sample.recipe.sample_surface_area:
                match5 = True
        match = match1 and match2 and match3 and match4 and match5
        if match:
            substratemap[s.Sample.id] = sb.id
            break
        else:
            substratemap[s.Sample.id] = None
    #################################################################################
    # Match to row(s) in Author table
    #################################################################################
    for oa in oauthors:
        if not oa.sample_id == s.Sample.id:
            continue
        for na in new_authors:
            match = match1 = match2 = False
            if oa.first_name == na.first_name:
                match1 = True
            if oa.last_name == na.last_name:
                match2 = True
            match = match1 and match2
            if match:
                authormap[s.Sample.id].append(na)
                break
        
        
    

#Print out the maps
print (f"fmap: {furnacemap}")
print (f"sbmap: {substratemap}")
print(f"amap: {authormap}")
print ("Done.")        

#####################################################################################
#####################################################################################

print("Adding Experiment rows...")
with db2.session_scope(autocommit=True) as ses2:
    for s in samples:
        print(f"{s.Sample.id}")
        try:
            carbon_source = s.Recipe.carbon_source
        except IndexError:
            carbon_source = None
        

            
        # Add the new experiment based on the old sample
        # Create a recipe for each old sample and add all of its prep steps
        newexp = Experiment2(
                id=s.Sample.id,
                furnace_id = furnacemap[s.Sample.id],
                substrate_id = substratemap[s.Sample.id],
                authors = authormap[s.Sample.id],
                material_name=s.Sample.material_name,
                experiment_date=s.Sample.experiment_date,
                recipe = Recipe2(
                    carbon_source=carbon_source,
                    base_pressure=s.Recipe.base_pressure,
                    preparation_steps = [
                        PreparationStep2(
                            name=step.name,
                            step=step.step,
                            duration=step.duration,
                            furnace_pressure=step.furnace_pressure,
                            furnace_temperature=step.furnace_temperature,
                            sample_location=step.sample_location,
                            helium_flow_rate=step.helium_flow_rate,
                            hydrogen_flow_rate=step.hydrogen_flow_rate,
                            carbon_source_flow_rate=step.carbon_source_flow_rate,
                            argon_flow_rate=step.argon_flow_rate,
                            cooling_rate=step.cooling_rate
                        ) for step in s.Recipe.preparation_steps
                    ]
                    ),
                
                )
        
        ses2.add(newexp)

print("done.")

#####################################################################################
#####################################################################################

print("Adding Environment Conditions rows...")
new_environment_conditions = [
    EnvironmentConditions2(
        # TODO: Where do we get ambient temperature
        dew_point = s.Sample.recipe.dewpoint,
        ambient_temperature = 10
    ) for s in samples
]
for s in samples:
    print(f"{s.Sample.recipe.dewpoint}")
    print(f"{s.Sample.id}")
print(new_environment_conditions)
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_environment_conditions)
        
#####################################################################################
print("Adding SemFile rows...")        
        
sem_files = ses1.query(SemFile).all()

new_sem_files = [
    SemFile2(
        id = f.id,
        experiment_id=f.sample_id,
        filename=f.filename,
        url=f.url
    )
    for f in sem_files
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_sem_files)
print("done.")  
      
#####################################################################################

print("Adding RamanFile rows...")        
        
raman_files = ses1.query(RamanFile).all()

new_raman_files = [
    RamanFile2(
        id = f.id,
        experiment_id=f.sample_id,
        filename=f.filename,
        url=f.url,
        wavelength=f.wavelength,
    )
    for f in raman_files
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_raman_files)
print("done.") 

#####################################################################################

print("Adding RamanSet rows to RamanAnalysis")        

raman_spectrum = ses1.query(RamanSpectrum).all()
raman_set = ses1.query(RamanSet).all()

new_raman_analyses_files = [
    RamanAnalysis2(
        id = f.id,
        raman_file_id=f.raman_file_id,
#         software_name=f.software_name,
#         software_version=f.software_version,
        d_peak_amplitude=f.d_peak_amplitude,
        d_peak_shift=f.d_peak_shift,
        d_fwhm=f.d_fwhm,
        g_peak_amplitude=f.g_peak_amplitude,
        g_peak_shift=f.g_peak_shift,
        g_fwhm=f.g_fwhm,
        g_prime_peak_amplitude=f.g_prime_peak_amplitude,
        g_prime_peak_shift=f.g_prime_peak_shift,
        g_prime_fwhm=f.g_prime_fwhm,        
        d_to_g = [q.d_to_g for q in raman_set if q.sample_id==f.set_id ],
        gp_to_g = [q.gp_to_g for q in raman_set if q.sample_id==f.set_id ]
    )

    for f in raman_spectrum
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_raman_analyses_files)
print("done.") 


Clearing destination database...
done.
Adding furnace rows...
done.
Adding substrate rows...
done.
Adding author rows...
done.
Getting old sample data...
done.
Matching parent tables...
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name: Graphene
name

Adding SemFile rows...
done.
Adding RamanFile rows...
done.
Adding RamanSet rows to RamanAnalysis
done.
